# Glasso Network Construction

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.covariance import GraphicalLasso

In [2]:
def clr_transform(data_matrix, pseudocount=10):
    """
    Performs a Centered Log-Ratio (CLR) transformation.
    Handles both Pandas DataFrames and Numpy arrays.
    """
    # Convert to numpy array if it is a pandas DataFrame/Series
    if hasattr(data_matrix, 'values'):
        data = data_matrix.values
    else:
        data = data_matrix

    # Add pseudocount to avoid log(0)
    data_plus_pseudo = data + pseudocount
    
    # Apply logarithm
    log_data = np.log(data_plus_pseudo)
    
    # Calculate the log of the geometric mean for each row (axis=1)
    # Using .reshape(-1, 1) to allow broadcasting during subtraction
    gmean_log = np.mean(log_data, axis=1).reshape(-1, 1)
    
    # CLR transform: log(x) - log(geometric_mean)
    clr_data = log_data - gmean_log
    
    return clr_data

In [3]:
# --- 0. FILE PATHS & CONFIGURATION ---
file_metalog = "../data/metalog_bgcs_with_gcf_and_tax.tsv"
file_mgnify = "../data/mgnify_bgcs_with_gcf_and_tax.tsv"
cols_to_use = ['analysis_accession', 'gcf_id'] 

print("Starting GLASSO Abundance Matrix Creation...")

# --- 1. DATA LOADING & CONCATENATION ---
try:
    print("Loading Metalog dataset...")
    df1 = pd.read_csv(file_metalog, sep=r'\t', usecols=cols_to_use)
    print("Loading MGnify dataset...")
    df2 = pd.read_csv(file_mgnify, sep=r'\t', usecols=cols_to_use)
    
    # Combine both datasets into a single dataframe
    df = pd.concat([df1, df2], ignore_index=True)
    print(f"Datasets combined. Total rows: {len(df)}")

except FileNotFoundError:
    print("ERROR: Files not found. Please check the paths!")
    raise

# --- 2. DATA CLEANING (Ensuring consistency across all notebooks) ---

# A. Drop rows where GCF-ID is NaN
df = df.dropna(subset=['gcf_id'])

# B. Normalize GCF IDs: Convert to string and remove ".0" suffix (fixes float conversion issues)
df['gcf_id'] = df['gcf_id'].astype(str).str.replace(r'\.0$', '', regex=True)

# C. Filter noise: Use a comprehensive list to catch all variants of unclustered/invalid IDs
noise_list = ["-1", "nan", "None", "", "unknown"]
df = df[~df["gcf_id"].isin(noise_list)]

print(f"Cleaned data: {len(df)} rows remaining.")
print(f"Unique GCFs identified: {df['gcf_id'].nunique()}")

# --- 3. MEMORY-EFFICIENT PRE-FILTERING (Crucial Step) ---

# A. Calculate prevalence per GCF (number of unique samples each GCF appears in)
gcf_prevalence = df.groupby('gcf_id')['analysis_accession'].nunique()
total_samples = df['analysis_accession'].nunique()

# B. Define 1% prevalence threshold
min_prevalence_percent = 0.01
min_samples = int(min_prevalence_percent * total_samples)

# C. Identify GCFs that pass the threshold
passing_gcfs = gcf_prevalence[gcf_prevalence >= min_samples].index
print(f"GCFs passing 1% threshold: {len(passing_gcfs)}")

# D. Filter the main dataframe BEFORE pivoting to avoid RAM overflow (Performance Warning)
df_filtered = df[df['gcf_id'].isin(passing_gcfs)]

# --- 4. CREATE ABUNDANCE MATRIX (Optimized) ---
# Group by sample and GCF to generate counts
df_counts = df_filtered.groupby(['analysis_accession', 'gcf_id']).size()

# Pivot the data (unstack) into a matrix (Samples as rows, GCFs as columns)
df_abundanz_final = df_counts.unstack(fill_value=0)
df_abundanz_final.index.name = 'assembly' 
print(f"GCFs vor Duplikat-Check: {len(df_abundanz_final.columns)}")

df_transposed = df_abundanz_final.T
df_final = df_transposed.drop_duplicates().T

# Wir benennen es wieder zurück auf df_abundanz_final, 
# damit der restliche Notebook-Code (CLR etc.) einfach weiterläuft.
df_abundanz_final = df_final 

print(f"GCFs nach Duplikat-Check: {len(df_abundanz_final.columns)}")
print(f"Abundance matrix created: {df_abundanz_final.shape[0]} samples x {df_abundanz_final.shape[1]} GCFs.")

Starting GLASSO Abundance Matrix Creation...
Loading Metalog dataset...


/tmp/ipykernel_3473280/796080711.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df1 = pd.read_csv(file_metalog, sep=r'\t', usecols=cols_to_use)


Loading MGnify dataset...


/tmp/ipykernel_3473280/796080711.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df2 = pd.read_csv(file_mgnify, sep=r'\t', usecols=cols_to_use)


Datasets combined. Total rows: 9741492
Cleaned data: 9741406 rows remaining.
Unique GCFs identified: 73520
GCFs passing 1% threshold: 1508
GCFs vor Duplikat-Check: 1508
GCFs nach Duplikat-Check: 1508
Abundance matrix created: 61622 samples x 1508 GCFs.


In [4]:

# --- 5. CLR TRANSFORMATION ---
# FIX: Pseudocount of 10 to stabilize compositions and handle zeros
# We pass the values to ensure the output is a clean numpy matrix for GLASSO
print("Applying Centered Log-Ratio (CLR) transformation...")
clr_data = clr_transform(df_abundanz_final, pseudocount=10)

# --- 6. NETWORK INFERENCE (GLASSO) ---
# Set alpha to 0.002 to prevent floating point errors often seen at 0.001
new_alpha = 0.002
print(f"Fitting GLASSO model with Alpha={new_alpha}...")

# Initialize and fit the Graphical Lasso model
# mode='cd' (Coordinate Descent) is generally faster for these dimensions
model = GraphicalLasso(alpha=new_alpha, mode='cd', tol=1e-4, max_iter=1000)
model.fit(clr_data)

# The result is the precision matrix (Omega / Inverse Covariance)
# This matrix represents direct conditional dependencies
precision_matrix = model.precision_
print("GLASSO model fitted successfully.")

Applying Centered Log-Ratio (CLR) transformation...
Fitting GLASSO model with Alpha=0.002...
GLASSO model fitted successfully.


In [5]:
# --- 7. BUILD GLASSO GRAPH (MIT FILTER) ---
print("Building GLASSO Graph...")
gcf_names = df_abundanz_final.columns
G_glasso = nx.Graph()
G_glasso.add_nodes_from(gcf_names)

# Zähler für die Statistik (damit du siehst, was passiert)
count_pos = 0
count_neg = 0

# Create graph from the precision matrix
for i in range(len(gcf_names)):
    for j in range(i + 1, len(gcf_names)): # Nur oberes Dreieck der Matrix
        
        raw_val = precision_matrix[i, j]
        
        if raw_val != 0:
            # SCHRITT 1: Vorzeichen umdrehen!
            # In der Präzisionsmatrix ist ein negativer Wert = positive Korrelation
            association = -raw_val 
            
            # SCHRITT 2: Filter auf NUR POSITIVE Werte
            # Wir wollen nur Kooperationen (Communities), keine Konkurrenz
            if association > 0:
                count_pos += 1
                
                # Kante hinzufügen mit dem korrekten positiven Gewicht
                G_glasso.add_edge(gcf_names[i], gcf_names[j], weight=association)
            else:
                count_neg += 1

# --- 8. STATISTIK AUSGEBEN ---
print("-" * 30)
print(f"GLASSO TEST ERGEBNIS (Alpha={new_alpha}):")
print(f"Gefundene Kooperationen (behalten): {count_pos}")
print(f"Gefundene Konkurrenzen (gelöscht): {count_neg}")
print(f"Finale Knoten im Graph: {G_glasso.number_of_nodes()}")
print(f"Finale Kanten im Graph: {G_glasso.number_of_edges()}")
print("-" * 30)

# --- 9. SAVE GRAPH ---
if G_glasso.number_of_edges() > 0:
    # Relabel nodes to strings for .gexf
    G_to_save = nx.relabel_nodes(G_glasso, str)
    output_filename = "../data/new_data_glasso_network_FILTERED2.gexf"
    nx.write_gexf(G_to_save, output_filename)
    print(f"Graph gespeichert als: {output_filename}")
else:
    print("⚠️ ACHTUNG: Der Graph ist leer! Versuch ein niedrigeres Alpha (z.B. 0.001).")


Building GLASSO Graph...
------------------------------
GLASSO TEST ERGEBNIS (Alpha=0.002):
Gefundene Kooperationen (behalten): 1376
Gefundene Konkurrenzen (gelöscht): 1288
Finale Knoten im Graph: 1508
Finale Kanten im Graph: 1376
------------------------------
Graph gespeichert als: ../data/new_data_glasso_network_FILTERED2.gexf
